In [85]:
import pandas as pd
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
import datetime
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
import pickle

In [77]:
stats = pd.read_csv("/Users/paramjaswal/Desktop/NBa/Player_mvp_stats.csv")
del stats["Unnamed: 0"]
stats

,Unnamed: 0,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,Pts Max,Share,Team,W,L,W/L%,GB,PS/G,PA/G,SRS
0,0,A.C. Green,PF,22.0,LAL,82.0,1.0,18.8,2.5,4.7,...,0.0,0.000,Los Angeles Lakers,62,20,0.756,0.0,117.3,109.5,6.84
1,1,Byron Scott,SG,24.0,LAL,76.0,62.0,28.8,6.7,13.0,...,0.0,0.000,Los Angeles Lakers,62,20,0.756,0.0,117.3,109.5,6.84
2,2,James Worthy,SF,24.0,LAL,75.0,73.0,32.7,8.4,14.5,...,780.0,0.009,Los Angeles Lakers,62,20,0.756,0.0,117.3,109.5,6.84
3,3,Jerome Henderson,C,26.0,LAL,1.0,0.0,3.0,2.0,3.0,...,0.0,0.000,Los Angeles Lakers,62,20,0.756,0.0,117.3,109.5,6.84
4,4,Kareem Abdul-Jabbar,C,38.0,LAL,79.0,79.0,33.3,9.6,16.9,...,780.0,0.173,Los Angeles Lakers,62,20,0.756,0.0,117.3,109.5,6.84
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22189,22189,Mike Gale,PG,29.0,SAS,67.0,0.0,22.0,2.6,5.6,...,0.0,0.000,San Antonio Spurs,41,41,0.500,9.0,119.4,119.7,-0.24
22190,22190,Paul Griffin,C,26.0,SAS,82.0,0.0,22.1,2.1,3.8,...,0.0,0.000,San Antonio Spurs,41,41,0.500,9.0,119.4,119.7,-0.24
22191,22191,Sylvester Norris,C,22.0,SAS,17.0,0.0,11.1,1.1,2.5,...,0.0,0.000,San Antonio Spurs,41,41,0.500,9.0,119.4,119.7,-0.24
22192,22192,Tim Bassett,PF,28.0,SAS,12.0,0.0,13.7,1.0,2.8,...,0.0,0.000,San Antonio Spurs,41,41,0.500,9.0,119.4,119.7,-0.24


In [4]:
stats.columns

Index(['Player', 'Pos', 'Age', 'Tm', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P',
       '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB',
       'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'Year',
       'Pts Won', 'Pts Max', 'Share', 'Team', 'W', 'L', 'W/L%', 'GB', 'PS/G',
       'PA/G', 'SRS'],
      dtype='object')

In [5]:
predictors = ['Age',  'G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P',
       '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB',
       'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'Year',
        'W', 'L', 'W/L%', 'GB', 'PS/G', 'PA/G', 'SRS']

In [6]:
train = stats[stats["Year"] < 2022]
test = stats[stats["Year"] == 2022]

In [7]:
reg = Ridge(alpha = 0.1)

In [8]:
reg.fit(train[predictors], train['Share'])

Ridge(alpha=0.1)

In [9]:
predictions = reg.predict(test[predictors])
predictions = pd.DataFrame(predictions, columns=['Predictions'], index=test.index)

In [10]:
predictions

,Predictions
727,0.014073
728,-0.033466
729,-0.003374
730,0.015952
731,-0.009124
...,...
18717,-0.018172
18718,-0.011725
18719,0.005339
18720,0.004157


In [11]:
combinations = pd.concat([test[["Player", "Share"]], predictions], axis=1)
combinations

,Player,Share,Predictions
727,Aaron Gordon,0.0,0.014073
728,Austin Rivers,0.0,-0.033466
729,Bol Bol,0.0,-0.003374
730,Bones Hyland,0.0,0.015952
731,Bryn Forbes,0.0,-0.009124
...,...,...,...
18717,Micah Potter,0.0,-0.018172
18718,Rodney McGruder,0.0,-0.011725
18719,Saben Lee,0.0,0.005339
18720,Saddiq Bey,0.0,0.004157


In [12]:
combinations.sort_values("Share", ascending=False).head(40)

,Player,Share,Predictions
742,Nikola Jokić,0.875,0.205691
916,Joel Embiid,0.706,0.189934
17762,Giannis Antetokounmpo,0.595,0.211422
986,Devin Booker,0.216,0.087892
17541,Luka Dončić,0.146,0.166332
1325,Jayson Tatum,0.043,0.100802
18435,Ja Morant,0.010,0.118898
8626,Stephen Curry,0.004,0.104570
984,Chris Paul,0.002,0.091914
10787,LeBron James,0.001,0.160361


In [13]:
mean_squared_error(combinations["Share"], combinations["Predictions"])

0.002277897532206951

In [14]:
combinations["Share"].value_counts()

0.000    593
0.001      3
0.875      1
0.706      1
0.002      1
0.216      1
0.043      1
0.004      1
0.146      1
0.595      1
0.010      1
Name: Share, dtype: int64

In [20]:
combinations = combinations.sort_values("Share", ascending=False)
combinations['Rk'] = list(range(1, combinations.shape[0] + 1))
combinations.head(20)

,Player,Share,Predictions,Rk,Predicted Rk
742,Nikola Jokić,0.875,0.205691,1,2
916,Joel Embiid,0.706,0.189934,2,3
17762,Giannis Antetokounmpo,0.595,0.211422,3,1
986,Devin Booker,0.216,0.087892,4,19
17541,Luka Dončić,0.146,0.166332,5,4
1325,Jayson Tatum,0.043,0.100802,6,14
18435,Ja Morant,0.010,0.118898,7,8
8626,Stephen Curry,0.004,0.104570,8,11
984,Chris Paul,0.002,0.091914,9,17
8413,Kevin Durant,0.001,0.144556,10,6


In [21]:
combinations = combinations.sort_values("Predictions", ascending=False)
combinations['Predicted Rk'] = list(range(1, combinations.shape[0] + 1))
combinations.head(20)

,Player,Share,Predictions,Rk,Predicted Rk
17762,Giannis Antetokounmpo,0.595,0.211422,3,1
742,Nikola Jokić,0.875,0.205691,1,2
916,Joel Embiid,0.706,0.189934,2,3
17541,Luka Dončić,0.146,0.166332,5,4
10787,LeBron James,0.001,0.160361,12,5
8413,Kevin Durant,0.001,0.144556,10,6
915,James Harden,0.000,0.124531,406,7
18435,Ja Morant,0.010,0.118898,7,8
17932,Trae Young,0.000,0.117510,407,9
10777,Anthony Davis,0.000,0.105644,408,10


In [22]:
# Average Precision

In [23]:
combinations = combinations.sort_values("Share", ascending=False)
combinations.head(20)

,Player,Share,Predictions,Rk,Predicted Rk
742,Nikola Jokić,0.875,0.205691,1,2
916,Joel Embiid,0.706,0.189934,2,3
17762,Giannis Antetokounmpo,0.595,0.211422,3,1
986,Devin Booker,0.216,0.087892,4,19
17541,Luka Dončić,0.146,0.166332,5,4
1325,Jayson Tatum,0.043,0.100802,6,14
18435,Ja Morant,0.010,0.118898,7,8
8626,Stephen Curry,0.004,0.104570,8,11
984,Chris Paul,0.002,0.091914,9,17
8413,Kevin Durant,0.001,0.144556,10,6


In [25]:
def avg_precision(combinations):
    actual = combinations.sort_values("Share", ascending=False).head(5)
    predicted = combinations.sort_values("Predictions", ascending=False)

    ps = []
    found = 0
    seen = 1
    for index, row in predicted.iterrows():
        if row["Player"] in actual["Player"].values:
            found += 1
            ps.append(found/seen)

        seen += 1
    return sum(ps)/len(ps)

avg_precision(combinations)

0.8526315789473685

In [26]:
# Backtesting to predict years

In [28]:
current_year = datetime.datetime.now().year
years = list(range(1956, current_year))

In [63]:
def add_ranks(combinations):
    combinations = combinations.sort_values("Share", ascending=False)
    combinations['Rk'] = list(range(1, combinations.shape[0] + 1))
    combinations = combinations.sort_values("Predictions", ascending=False)
    combinations['Predicted Rk'] = list(range(1, combinations.shape[0] + 1))
    combinations["Difference"] = combinations["Predicted Rk"] - combinations["Rk"]
    return combinations

In [51]:
def backtesting(stats, model, years, predictors):
    avg_p = []
    all_predictions = []
    for year in years[5:]:
        train = stats[stats["Year"] < year]
        test = stats[stats["Year"] == year]
        model.fit(train[predictors], train["Share"])
        predictions = reg.predict(test[predictors])
        predictions = pd.DataFrame(predictions, columns=['Predictions'], index=test.index)
        combinations = pd.concat([test[["Player", "Share"]], predictions], axis=1)
        combinations = add_ranks(combinations)
        all_predictions.append(combinations)
        avg_p.append(avg_precision(combinations))
    return sum(avg_p)/len(avg_p), avg_p, pd.concat(all_predictions)



mean_avg_p, avg_p, all_predictions = backtesting(stats, reg, years, predictors)

In [53]:
mean_avg_p

0.7483113014816261

In [55]:
all_predictions[all_predictions['Rk'] <= 5].sort_values("Difference", ascending=False).head(20)

,Player,Share,Predictions,Rk,Predicted Rk,Difference
21811,Dennis Johnson,0.009,0.015418,5,66,61
11220,Glen Rice,0.117,0.028179,5,57,52
14888,Bernard King,0.491,0.047402,2,31,29
1645,Chris Paul,0.138,0.079411,5,30,25
11488,Peja Stojaković,0.228,0.041570,4,28,24
1060,Pete Maravich,0.061,0.053195,3,23,20
6989,Sidney Moncrief,0.301,0.060797,4,23,19
15062,Sam Jones,0.040,0.047812,4,23,19
20046,Joakim Noah,0.258,0.064595,4,22,18
14187,Spencer Haywood,0.054,0.068730,5,23,18


In [57]:
pd.concat([pd.Series(reg.coef_), pd.Series(predictors)], axis=1).sort_values(0, ascending=False)

,0,1
13,0.058209,eFG%
29,0.034463,W/L%
10,0.021339,2P
22,0.012816,BLK
21,0.011370,STL
7,0.008545,3P
20,0.008512,AST
19,0.008161,TRB
25,0.008096,PTS
12,0.007978,2P%


In [59]:
# Random Forest Regressor

In [60]:
rf = RandomForestRegressor(n_estimators=50, random_state=1, min_samples_split=5)

In [61]:
mean_avg_p, avg_p, all_predictions = backtesting(stats, rf, years, predictors)

In [62]:
mean_avg_p

0.7756968936311986

In [64]:
### model for flask

In [65]:
# Sort the data by Year
sorted_data = stats.sort_values(by="Year")

# Split the data
train_size = int(0.8 * len(sorted_data))
train_data = sorted_data[:train_size]
test_data = sorted_data[train_size:]

In [66]:
X_train = train_data[predictors]
y_train = train_data["Share"]

rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

RandomForestRegressor(random_state=42)

In [68]:
X_test = test_data[predictors]
y_test = test_data["Share"]

predictions = rf_model.predict(X_test)
mae = mean_absolute_error(y_test, predictions)
print("Mean Absolute Error:", mae)

Mean Absolute Error: 0.006105539535931516


In [70]:
# Define the hyperparameters and their possible values
param_grid = {
    'n_estimators': [10, 50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False]
}

# Initialize Random Forest Regressor
rf = RandomForestRegressor()

# Set up the grid search
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, 
                           cv=3, n_jobs=-1, verbose=2, scoring='neg_mean_absolute_error')

In [71]:
# Splitting the data again for clarity
X = stats[predictors]
y = stats["Share"]

# Fit the grid search to the data
grid_search.fit(X, y)

# Get the best parameters from the grid search
best_params = grid_search.best_params_
print(f"Best Parameters: {best_params}")

Fitting 3 folds for each of 288 candidates, totalling 864 fits
[CV] END bootstrap=True, max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=10; total time=   2.9s
[CV] END bootstrap=True, max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=10; total time=   2.9s
[CV] END bootstrap=True, max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=10; total time=   2.9s
[CV] END bootstrap=True, max_depth=None, min_samples_leaf=1, min_samples_split=5, n_estimators=10; total time=   2.7s
[CV] END bootstrap=True, max_depth=None, min_samples_leaf=1, min_samples_split=5, n_estimators=10; total time=   2.7s
[CV] END bootstrap=True, max_depth=None, min_samples_leaf=1, min_samples_split=5, n_estimators=10; total time=   2.8s
[CV] END bootstrap=True, max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=  13.9s
[CV] END bootstrap=True, max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time= 

In [72]:
best_rf = RandomForestRegressor(**best_params)
best_rf.fit(X_train, y_train)
predictions = best_rf.predict(X_test)

mae = mean_absolute_error(y_test, predictions)
print(f"Mean Absolute Error: {mae}")

Mean Absolute Error: 0.00584379000903414


In [74]:
X_all = sorted_data[predictors]
y_all = sorted_data["Share"]

rf_model_final = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model_final.fit(X_all, y_all)

RandomForestRegressor(random_state=42)

In [88]:
# Save the model to a pickle file
with open('/Users/paramjaswal/Desktop/NBa/rf_mvp_model.pkl', 'wb') as file:
    pickle.dump(rf_model, file)

In [100]:
# Load the model from the pickle file
with open('/Users/paramjaswal/Desktop/NBa/rf_mvp_model.pkl', 'rb') as file:
    loaded_model = pickle.load(file)

# Check the type of the loaded model
print(type(loaded_model))

<class 'sklearn.ensemble._forest.RandomForestRegressor'>


In [131]:
def predict_mvp_share(player_stats, model):
    # Ensure the input data is in DataFrame format
    data = pd.DataFrame([player_stats])
    
    # Use the model to predict MVP Share
    predicted_share = model.predict(data)
    return predicted_share[0]

In [132]:
player_name = input("Enter the name of the hypothetical player: ")

# Hypothetical player's stats
player_data = {
    'Age': 25,  # Prime age
    'G': 82,  # Played all games
    'GS': 82,  # Started all games
    'MP': 40.0,  # High minutes per game
    'FG': 24.0,  # High field goals made
    'FGA': 23.0,  # Efficient shooting
    'FG%': 0.90,  # Very high field goal percentage
    '3P': 9.0,  # Elite 3-point shooter
    '3PA': 10.0,  # Good 3-point attempts
    '3P%': 0.625,  # Elite 3-point percentage
    '2P': 9.0,  # Good 2-point field goals made
    '2PA': 16.0,  # Efficient 2-point shooting
    '2P%': 0.583,  # High 2-point percentage
    'eFG%': 0.97,  # Extremely high effective field goal percentage
    'FT': 10.0,  # High free throws made
    'FTA': 12.0,  # High free throw attempts
    'FT%': 0.90,  # Excellent free throw percentage
    'ORB': 6,  # Offensive rebounds
    'DRB': 15.0,  # Defensive rebounds
    'TRB': 20,  # Total rebounds
    'AST': 11.0,  # Elite assists
    'STL': 2.5,  # Elite steals
    'BLK': 6,  # Good blocks for a guard/forward
    'TOV': 1.0,  # Low turnovers
    'PF': 1.0,  # Low personal fouls
    'PTS': 37.0,  # Elite scoring
    'Year': 2022,
    'W': 70,  # Team won a lot of games
    'L': 12,  # Few losses
    'W/L%': 0.850,  # High win-loss percentage
    'GB': 1,  # Close to top seed
    'PS/G': 120.0,  # Team's high points per game
    'PA/G': 100.0,  # Team's low points allowed per game
    'SRS': 10.0  # High simple rating system
}




print(f"Predicted MVP Share for the {player_name} in the given year: {predict_mvp_share(player_data, loaded_model)}")

Predicted MVP Share for the Leo in the given year: 0.8598600000000002


In [125]:
def get_mvp_rank(predicted_share, year, data):
    # Filter the dataset for the specific year
    year_data = data[data['Year'] == year]
    
    # Extract the MVP shares for all players in that year
    mvp_shares = year_data['Share'].tolist()
    
    # Add the predicted MVP share of the hypothetical player
    mvp_shares.append(predicted_share)
    
    # Sort the list in descending order
    mvp_shares.sort(reverse=True)
    
    # Find the rank of the hypothetical player
    rank = mvp_shares.index(predicted_share) + 1
    
    return rank

# Assuming you've already predicted the MVP share for the hypothetical player
predicted_share = predict_mvp_share(player_data, loaded_model)
year = 2022  # You can change this to any year you want
rank = get_mvp_rank(predicted_share, year, stats)

print(f"Predicted MVP Rank for the {player_name} for the year {year}: {rank}")

Predicted MVP Rank for the Mohit for the year 2022: 2


In [126]:
def get_mvp_neighbors(predicted_share, year, data, player_name='Hypothetical Player', n=5):
    player_name = input("Enter the name of the hypothetical player: ")

    # Filter the dataset for the specific year
    year_data = data[data['Year'] == year]
    
    # Extract the MVP shares and player names for all players in that year
    mvp_shares = year_data[['Player', 'Share']]
    
    # Add the predicted MVP share of the hypothetical player
    mvp_shares = mvp_shares.append({'Player': player_name, 'Share': predicted_share}, ignore_index=True)
    
    # Sort the dataframe in descending order based on MVP shares
    mvp_shares = mvp_shares.sort_values(by='Share', ascending=False).reset_index(drop=True)
    
    # Find the rank of the hypothetical player
    rank = mvp_shares[mvp_shares['Player'] == player_name].index[0]
    
    # Get the neighboring players
    start_index = max(0, rank - n)
    end_index = min(len(mvp_shares), rank + n + 1)
    neighbors = mvp_shares.iloc[start_index:end_index]
    
    return neighbors

# Assuming you've already predicted the MVP share for the hypothetical player
predicted_share = predict_mvp_share(player_data, loaded_model)
year = 2022  # You can change this to any year you want
player_name = "John Doe"  # Replace with the player's name
neighbors_table = get_mvp_neighbors(predicted_share, year, stats, player_name)

print(neighbors_table)

                  Player    Share
0           Nikola Jokić  0.87500
1                  Mohit  0.85986
2            Joel Embiid  0.70600
3  Giannis Antetokounmpo  0.59500
4           Devin Booker  0.21600
5            Luka Dončić  0.14600
6           Jayson Tatum  0.04300


/var/folders/dy/0p2_mb093ps8390h87cvhc6m0000gn/T/ipykernel_48103/2650383098.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mvp_shares = mvp_shares.append({'Player': player_name, 'Share': predicted_share}, ignore_index=True)
